In [ ]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import cifar10,cifar100

In [ ]:
from . SpatialReasoningModel import *

In [ ]:
# example of loading the mnist dataset
from numpy import mean
from numpy import std
from sklearn import datasets
from sklearn.manifold import TSNE
from matplotlib import pyplot as plt
from sklearn.model_selection import KFold
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten, BatchNormalization
from tensorflow.keras.optimizers import SGD
import tensorflow as tf

from tensorflow.keras.utils import to_categorical
#from tensorflow.keras.datasets import cifar10,cifar100
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import keras
import numpy as np
def conc(*inp1):
  return layers.Concatenate()(inp1)

#!pip install tensorflow_addons
import tensorflow_addons as tfa

def mpool(psize,strides=2):
  return MaxPooling2D(pool_size=psize,strides=strides,padding="SAME")

def apool(psize,strides=None):
  if(strides is None):
    return AveragePooling2D(pool_size=psize,padding="SAME")
  else:
    return AveragePooling2D(pool_size=psize,strides=strides,padding="SAME")

def ln():
  return layers.LayerNormalization()

def bn():
  return layers.BatchNormalization()

def dense(size,act='relu'):
  return Dense(size,activation=act)

from keras.callbacks import ModelCheckpoint
import tensorflow.keras.layers as layers
filepath = 'my_best_model.hdf5'

def bnconv(inp,units,kernel_size):
  return BatchNormalization()(Conv2D(units,kernel_size,activation='relu',kernel_regularizer=tf.keras.regularizers.L2(0.0005))(inp))

def dense(units,act='relu'):
  return layers.Dense(units,activation=act)

def edim(inp,axis=-1):
    return tf.expand_dims(inp,axis)

def conv(inp,units,kernel_size):
  return Conv2D(kernel_size,units,activation='relu',kernel_regularizer=tf.keras.regularizers.L2(0.0005))(inp)

def cutout(image,size):
  return tfa.image.cutout(image,mask_size=(size,size))

def LaplacianFilter(input_tensor):
  filter = tf.constant([[[0.0,-1.0,0.0],[-1.0,4.0,-1.0],[0.0,-1.0,0.0]]])   #The filters for derivatives Cx, Cy ,[[0.0,-1.0],[1.0,0.0]]
  filter = filter[:,:,:,tf.newaxis]
  filter = tf.transpose(filter,[1,2,0,3])
  print(filter.shape)
  tf.print(filter[:,:,1,1])
  out = tf.nn.conv2d(input_tensor,filters=filter,padding="SAME",strides=[1, 1, 1, 1])  #basic convolution operation in tensorflow, the derivative filters are applied with stride 3
  return out


def LocalCurvature(input_tensor,scale=1,edge=True):
  filter = tf.constant([[[-1.0,0.0],[0.0,0.0]],[[1.0,0.0],[1.0,-1.0]]])   #The filters for derivatives Cx, Cy ,[[0.0,-1.0],[1.0,0.0]]
  filter = tf.transpose(filter,[1,2,0])
  filter = filter[:,:,tf.newaxis] #Expanding according to number of input channels
  filter = tf.tile(filter,[1,1,input_tensor.shape[-1],1])
  filter2 = tf.constant([[[0.0,0.0],[0.0,0.0]],[[1.0,0.0],[0.0,1.0]]])   #The filters for derivatives Cx, Cy ,[[0.0,1.0],[0.0,0.0]]
  filter2 = tf.transpose(filter2,[1,2,0])

  filter2 = filter2[:,:,tf.newaxis]
  filter2 = tf.tile(filter2,[1,1,input_tensor.shape[-1],1])                                         #Expanding according to number of input channels
  filter3 = tf.constant([[[1.0,0.0],[0.0,0.0]],[[0.0,0.0],[1.0,0.0]]])   #The filters for derivatives Cx, Cy ,[[0.0,0.0],[1.0,0.0]
  filter3 = tf.transpose(filter3,[1,2,0])
  filter3 = filter3[:,:,tf.newaxis]
  filter3 = tf.tile(filter3,[1,1,input_tensor.shape[-1],1])
  out2 = tf.nn.conv2d(input_tensor, filters=filter2,padding="SAME",dilations=scale,strides=[1, 1, 1, 1])
  out3 = tf.nn.conv2d(input_tensor, filters=filter3,padding="SAME",dilations=scale,strides=[1, 1, 1, 1])
  out = tf.math.abs(tf.nn.conv2d(input_tensor,filters=filter,dilations=scale,padding="SAME",strides=[1, 1, 1, 1]))  #basic convolution operation in tensorflow, the derivative filters are applied with stride 3
  if edge:
    return tf.where(tf.abs(out2*out3)>0,out,0.0)
  else:
    return out

def LocalCurvature(input_tensor,scale=1,edge=True):
  filter = tf.constant([[[-1.0,0.0],[1.0,0.0]],[[0.0,0.0],[1.0,-1.0]],[[0.0,-1.0],[1.0,0.0]]])   #The filters for derivatives Cx, Cy
  filter = filter[:,:,tf.newaxis]                                         #Expanding according to number of input channels
  filter = tf.tile(filter,[1,1,1,1])
  filter2 = tf.constant([[[0.0,0.0],[1.0,0.0]],[[0.0,0.0],[0.0,1.0]],[[0.0,1.0],[0.0,0.0]]])   #The filters for derivatives Cx, Cy
  filter2 = filter2[:,:,tf.newaxis]                                         #Expanding according to number of input channels
  filter3 = tf.constant([[[1.0,0.0],[0.0,0.0]],[[0.0,0.0],[1.0,0.0]],[[0.0,0.0],[1.0,0.0]]])   #The filters for derivatives Cx, Cy
  filter3 = filter3[:,:,tf.newaxis]
  filter2 = tf.tile(filter2,[1,1,1,1])
  out2 = tf.nn.conv2d(input_tensor, filters=filter2,padding="SAME",dilations=scale,strides=[1, 1, 1, 1])
  out3 = tf.nn.conv2d(input_tensor, filters=filter3,padding="SAME",dilations=scale,strides=[1, 1, 1, 1])
  out = tf.math.abs(tf.nn.conv2d(input_tensor,filters=filter,dilations=scale,padding="SAME",strides=[1, 1, 1, 1]))  #basic convolution operation in tensorflow, the derivative filters are applied with stride 3
  if edge:
    return tf.where(tf.abs(out2*out3)>0,out,0.0)
  else:
    return out

import tensorflow_datasets as tfds
import numpy as np

def preprocess_imagecont(image, label,image_size=28):
  image = tf.convert_to_tensor(image)
  image = tf.image.resize(image, [image_size,image_size])
  image1 = tfa.image.gaussian_filter2d(image, (2,2),3)
  image = tf.cast(image, tf.float32)
  #image = image/255.0
  positionsx1 = tf.range(start=0, limit=image_size, delta=float(1),dtype=tf.float32)
  positionsy1 = tf.range(start=0, limit=image_size, delta=float(1),dtype=tf.float32)
  positionsx1 = tf.expand_dims(tf.tile(tf.expand_dims(positionsx1,0),[image_size,1]),-1)
  positionsy1 = tf.expand_dims(tf.tile(tf.transpose(tf.expand_dims(positionsy1,0)),[1,image_size]),-1)
  positions11 = tf.concat([positionsx1,positionsy1],-1); positions11 = tf.tile(positions11[tf.newaxis,:,:,:],[image.shape[0],1,1,1])
  u = tf.image.sobel_edges(image1)
  angle = tf.where(u[:,:,:,0,0]!=0,tf.atan2(u[:,:,:,0,1],u[:,:,:,0,0]),0)
  angle = angle[:,:,:,tf.newaxis]
  #u = tf.reshape(u,[-1,image_size,image_size,6])

  image = tf.concat([image,angle/3.14,positions11],-1)
  #image = tf.concat([u[tf.newaxis,:,:,:],angle,positions11[tf.newaxis,:]],-1)
  #image=tf.squeeze(image,0)
  return image, label

labeled_batch_size=75
num_epochs = 60
batch_size = 30  # Corresponds to 200 steps per epoch
width = 28
temperature = 1
learning_rate=0.0001
#lr_drop=20


import keras.backend as K

#select = tf.convert_to_tensor(select,dtype=tf.float32)

def train(model,path,epochs=100):
    checkpoint1 = ModelCheckpoint(filepath='./'+path,save_format=tf,monitor='val_loss',
                            save_weights_only=True,
                             verbose=1,
                             save_best_only=True,
                             mode='min')
    model.compile(optimizer=tf.keras.optimizers.SGD(0.1,momentum=0.9),
    loss='categorical_crossentropy',
    metrics='categorical_accuracy')
    model.fit(trainXA, trainY, epochs=epochs, batch_size=200, validation_data=(testXA, testY), callbacks=[checkpoint1, reduce_lr], verbose=1)

learning_rate = 0.1
lr_drop=15
def lr_scheduler(epoch):
        return learning_rate * (0.5 ** (epoch // lr_drop))

reduce_lr = keras.callbacks.LearningRateScheduler(lr_scheduler)

checkpoint_gen = ModelCheckpoint(filepath='./'+'generative_cifar_2',save_format=tf,monitor='val_val_loss',
                            save_weights_only=True,
                             verbose=1,
                             save_best_only=True,
                             mode='min')
image_size=28

In [ ]:
image_size=28

In [ ]:
def load_dataset():
	# load dataset
  (trainX, trainY), (testX, testY) = mnist.load_data()
  trainX = trainX.reshape((trainX.shape[0], 28, 28, 1))
  testX = testX.reshape((testX.shape[0], 28, 28, 1))
  return trainX, trainY, testX, testY

def prep_pixels(train, test):
	# convert from integers to floats
  train_norm = train.astype('float32')
  test_norm = test.astype('float32')
	# normalize to range 0-1
  train_norm = train_norm / 255.0
  test_norm = test_norm / 255.0
  normalization = layers.Normalization()
  normalization.adapt(train_norm)
  #train_norm=normalization(train_norm)
  #test_norm=normalization(test_norm)
  return train_norm, test_norm

trainX, trainY, testX, testY = load_dataset()
	# prepare pixel data
trainX, testX = prep_pixels(trainX, testX)
trainXA,trainY = preprocess_imagecont(trainX,trainY)
testXA,testY = preprocess_imagecont(testX,testY)
#train_images_res = tf.image.resize(trainXA,[48,48],method='bilinear')
#test_images_res = tf.image.resize(testXA,[48,48],method='bilinear')

trainY = tf.keras.utils.to_categorical(trainY,num_classes=10)
testY = tf.keras.utils.to_categorical(testY,num_classes=10)

In [ ]:
testXA.shape

TensorShape([10000, 28, 28, 4])

In [ ]:
import math

In [ ]:
learning_rate = 0.0001
lr_drop=20
def lr_scheduler(epoch):
        return learning_rate * (0.5 ** (epoch // lr_drop))

reduce_lr = keras.callbacks.LearningRateScheduler(lr_scheduler)

checkpoint_gen = ModelCheckpoint(filepath='./'+'generative_cifar_2',save_format=tf,monitor='val_val_loss',
                            save_weights_only=True,
                             verbose=1,
                             save_best_only=True,
                             mode='max')

In [ ]:
def train(model,path,epochs=80):
  checkpoint1 = ModelCheckpoint(filepath='./'+path,save_format=tf,monitor='val_categorical_accuracy',
                            save_weights_only=True,
                             verbose=1,
                             save_best_only=True,
                             mode='max')
  model.compile(optimizer=tf.keras.optimizers.Adam(0.0001),
    loss='categorical_crossentropy',
    metrics='categorical_accuracy')
  model.fit(trainXA,trainY, epochs=epochs, batch_size=30, validation_data=(testXA,testY), callbacks=[reduce_lr,checkpoint1], verbose=1)

**Load CNN Model**

In [ ]:
c = ConvolutionalModel()
c.load_weights('conv_4_mnist')

**Invariance of CNN model**

In [ ]:
test_invariance(c)

original accuracy
99.45000410079956%
Accuracy on Images scaled by factor:
1.1
97.54000306129456%
1.2
81.6800057888031%
1.3
55.250000953674316%
1.4
45.00000178813934%
Accuracy on Images sheared by factor:
0.1
97.76000380516052%
0.2
87.55000233650208%
0.4
27.100002765655518%
0.5
13.220000267028809%


In [ ]:
srn = SpatialRelationsModel()()
train(srn,'2_lyr_srn_1conv_bn')

(None, 7, 7, 840)
inputsshape(None, 7, 7, 13440)
(None, 1240)
Epoch 1/80
(30, 7, 7, 840)
inputsshape(30, 7, 7, 13440)
(30, 1240)
(30, 7, 7, 840)
inputsshape(30, 7, 7, 13440)
(30, 1240)


2024-02-04 13:23:45.657333: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_2/tf.where_4/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


1998/2000 [============================>.] - ETA: 0s - loss: 1.2575 - categorical_accuracy: 0.9335(None, 7, 7, 840)
inputsshape(None, 7, 7, 13440)
(None, 1240)


2024-02-04 13:24:36.997778: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_2/tf.where_4/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer



Epoch 1: val_categorical_accuracy improved from -inf to 0.96500, saving model to ./2_lyr_srn_1conv_bn
2000/2000 [==============================] - 57s 26ms/step - loss: 1.2572 - categorical_accuracy: 0.9335 - val_loss: 1.0078 - val_categorical_accuracy: 0.9650 - lr: 1.0000e-04
Epoch 2/80
1998/2000 [============================>.] - ETA: 0s - loss: 0.8909 - categorical_accuracy: 0.9711
Epoch 2: val_categorical_accuracy improved from 0.96500 to 0.97110, saving model to ./2_lyr_srn_1conv_bn
2000/2000 [==============================] - 51s 26ms/step - loss: 0.8909 - categorical_accuracy: 0.9710 - val_loss: 0.7950 - val_categorical_accuracy: 0.9711 - lr: 1.0000e-04
Epoch 3/80
1999/2000 [============================>.] - ETA: 0s - loss: 0.6857 - categorical_accuracy: 0.9773
Epoch 3: val_categorical_accuracy improved from 0.97110 to 0.98020, saving model to ./2_lyr_srn_1conv_bn
2000/2000 [==============================] - 51s 26ms/step - loss: 0.6858 - categorical_accuracy: 0.9773 - val_loss

KeyboardInterrupt: 

In [ ]:
m64.evaluate(testXA,testY)

313/313 [==============================] - 13s 32ms/step - loss: 0.0554 - categorical_accuracy: 0.9907


[0.05539818853139877, 0.9907000660896301]

***Testing Invariance of Single layer SRN with 2 convolutional layers***

In [ ]:
srn.load_weights('single_layer_2conv_srn')

In [ ]:
test_invariance(srn)

original accuracy
97.8600025177002%
Accuracy on Images scaled by factor:
1.1
97.53000736236572%
1.2
97.23000526428223%
1.3
96.24000191688538%
1.4
95.65000534057617%
Accuracy on Images sheared by factor:
0.1
97.55000472068787%
0.2
96.17000222206116%
0.4
78.55000495910645%
0.5
64.26000595092773%


In [ ]:
test_invariance(m64)

313/313 [==============================] - 10s 32ms/step - loss: 0.0554 - categorical_accuracy: 0.9907
[0.05539818853139877, 0.9907000660896301]
313/313 [==============================] - 10s 31ms/step - loss: 0.0690 - categorical_accuracy: 0.9877
[0.06896796077489853, 0.9877000451087952]
313/313 [==============================] - 10s 31ms/step - loss: 0.0913 - categorical_accuracy: 0.9801
[0.0913410410284996, 0.9801000356674194]
313/313 [==============================] - 10s 32ms/step - loss: 0.2153 - categorical_accuracy: 0.9480
[0.21528370678424835, 0.9480000734329224]
313/313 [==============================] - 10s 32ms/step - loss: 0.7104 - categorical_accuracy: 0.8335
[0.7104433178901672, 0.8335000276565552]
313/313 [==============================] - 10s 32ms/step - loss: 3.6673 - categorical_accuracy: 0.3814
[3.667268991470337, 0.3814000189304352]
313/313 [==============================] - 10s 31ms/step - loss: 6.2965 - categorical_accuracy: 0.1795
[6.296536445617676, 0.179500013

In [ ]:
import math

In [ ]:
def edim(inp,axis=-1):
    return tf.expand_dims(inp,axis)

In [ ]:
import math

def test_invariance(model):
  print('original accuracy')
  print(str(model.evaluate(testXA,testY,verbose=0)[1]*100)+'%')
  print('Accuracy on Images scaled by factor:')
  print('1.1')
  print(str(affine_scale(model,1.1)[1]*100)+'%')
  print('1.2')
  print(str(affine_scale(model,1.2)[1]*100)+'%')
  print('1.3')
  print(str(affine_scale(model,1.3)[1]*100)+'%')
  print('1.4')
  print(str(affine_scale(model,1.4)[1]*100)+'%')
  affine_scale(model,2.0)
  print('Accuracy on Images sheared by factor:')
  print('0.1')
  print(str(affine_shear(model,0.1)[1]*100)+'%')
  print('0.2')
  print(str(affine_shear(model,0.2)[1]*100)+'%')
  print('0.4')
  print(str(affine_shear(model,0.4)[1]*100)+'%')
  print('0.5')
  print(str(affine_shear(model,0.5)[1]*100)+'%')

def affine_rot(model,theta):
    tr = tfa.image.rotate(testXA,(theta/180)*math.pi)
    image2 = tr
    #print(model.evaluate(testXA,testY))
    #indices = tf.where(tf.argmax(model(test_images),-1)==tf.argmax(test_labels),-1)
    #print(model.evaluate(image2[indices],test_labels[indices]))
    print(model.evaluate(tr,testY,verbose=0))

def affine_scale(model,scale):
    tr =  tfa.image.transform(testXA,[scale,0.0,0.0,0.0,scale,0,0,0])
    image2=tr
    return model.evaluate(image2,testY,verbose=0)

def affine_shear(model,scale):
    total_acc=0
    n=0
    #predictions1 = model.predict(testXA)
    #image2 =np.array(image2)
    tr = tfa.image.transform(testXA,[1.0,scale,0,scale,1.0,0,0,0])#tfa.image.shear_x(testXA,scale,replace=0.0) #
    #print(model.evaluate(testXA,testY))
    return model.evaluate(tr,testY,verbose=0)